In [ ]:
# unfreeze a few of the top layers of a frozen model base used for feature extraction and jointly train the new added model and these top layers
# the more specialized feature need in fact to be retrained

In [ ]:
from keras.applications import VGG16 # TODO: try also Xception, Inception V3, ResNet50, VGG16, VGG19, MobileNet and others domain-specific nets
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
images_path = os.path.join("..","workspace","images")
train_dir = os.path.join(images_path, "train")
validation_dir = os.path.join(images_path, "eval")
test_dir = os.path.join(images_path, "test")
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [ ]:
# hyperparameters to be set
input_height = 1920
input_width = 1080
# others

In [ ]:
conv_base = VGG16(weights='imagenet', # weight checkpoint from which we initialize the model
                  include_top=False, # to decide if need to include the densely connectedd layer
                  input_shape=(input_width, input_height, 3)) #TODO: check order

In [ ]:
conv_base.summary()

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1': # TODO: decide the layer where to stop
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# use optimizer with very low learning rate to limit the magnitude of modifications we make

In [ ]:
loss_function='mse'
metrics=['mae']
optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=5
num_classes=8
batch_size=20
if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest') # set right parameters

df_train = pd.read_csv(os.path.join(train_dir, "df_train.csv"))
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=metrics)

history = model.fit_generator(
      train_generator,
      steps_per_epoch=10,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=5)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

# if performances are much wors than validation ones, during hyperparameter optimization (when done) the process has overfitted the validdation set, if so go to a more clear protocol such as Kfold CV
test_loss, test_acc = model.evaluate_generator(test_generator, steps=2)
print('test acc:', test_acc)
print('test loss:', test_loss)